In [22]:
import pandas as pd
from IPython.display import clear_output
import requests
import os

#pip install eiapy

os.environ['EIA_KEY'] = 'pGSYm7fJwZOAuwgo5jGqrOG8wsanmMFVYpaxeZSJ'
API_KEY = os.environ['EIA_KEY']

from eiapy import Category, Series

import time

iid = 'EBA.TEX-ALL.D.H'
url = 'https://api.eia.gov/series/?api_key={}&series_id={}'.format(API_KEY, iid)
print (url)
s = requests.Session()
req = s.get(url)
json_data = req.json()
#print (json_data)


## GET EIA DATA
iid = 'EBA.TEX-ALL.D.H'
ser = Series(iid)
#print (ser.get_data(all_data=True))
#df = pd.DataFrame(ser.get_data(all_data=True)['series'][0]['data'])
df = pd.DataFrame(json_data['series'][0]['data'])
df.columns = ['dateraw', 'demand_mgwhr']
df['date'] = pd.to_datetime(df['dateraw'], format='%Y%m%dT%HZ')
df['hour_zulu'] = [x.hour for x in df['date']]
df['year'] = [x.year for x in df['date']]
del df['dateraw']
df['state'] = 'Texas'
df['dt_loaded'] = pd.Timestamp.date(pd.Timestamp.now())
df = df[['date','hour_zulu','state','demand_mgwhr','dt_loaded','year']]

print (df.head())



https://api.eia.gov/series/?api_key=pGSYm7fJwZOAuwgo5jGqrOG8wsanmMFVYpaxeZSJ&series_id=EBA.TEX-ALL.D.H
                 date  hour_zulu  state  demand_mgwhr   dt_loaded  year
0 2022-03-07 18:00:00         18  Texas         46367  2022-03-07  2022
1 2022-03-07 17:00:00         17  Texas         46753  2022-03-07  2022
2 2022-03-07 16:00:00         16  Texas         46644  2022-03-07  2022
3 2022-03-07 15:00:00         15  Texas         45948  2022-03-07  2022
4 2022-03-07 14:00:00         14  Texas         44807  2022-03-07  2022


In [21]:


#DB Connection

import mysql.connector
from mysql.connector.constants import ClientFlag

config = {
    'user': 'demouser',
    'password': 'demopassword',
    'host': '10.85.176.4',
#    'client_flags': [ClientFlag.SSL],
#    'ssl_ca': 'ssl/server-ca.pem',
#    'ssl_cert': 'ssl/client-cert.pem',
#    'ssl_key': 'ssl/client-key.pem'
}

# now we establish our connection
cnxn = mysql.connector.connect(**config)

config['database'] = 'power'  # add new database to config dict
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

drop_recent = "delete from power_demand where date >'2021-01-01 00:00:00'"

cursor.execute(drop_recent)

#Get Latest Record from Table to truncate what is added
latest_query=("select max(date) from power_demand")
try:
	cursor.execute(latest_query)
	last_update = cursor.fetchone()
	last_update =last_update[0].strftime('%Y-%m-%d %H:%M:%S')
	#print ("Last Updated:", last_update)
	print ("Last Record in DB:",last_update)
except:
	pass #no results in table
	last_update = '2000-01-01'

	print ("NO RECORDS in DB:",last_update)



#Insert New Records

## Trim Dataframe to new records
#print (df[df['date'] > last_update])
df = df[(df['date'] > last_update)]

print ("New Records:", len(df))


import time
df = df.sort_values(by='date', ascending=True)
for index, row in df.iterrows():
	#print(row['date'], row['hour_zulu'],row['state'],row['demand_mgwhr'],row['dt_loaded'])
	insert_query = ("INSERT INTO power_demand (date, hour_zulu, state, demand_mgwhr, dt_loaded)"
		" VALUES ('%s', '%s',' %s', '%s', '%s')" %( row['date'], row['hour_zulu'], row['state'], row['demand_mgwhr'], row['dt_loaded']))
	#print (insert_query)
	if row['year'] >= 2022:
		time.sleep(1)
		print ('Inserting Slowly...', insert_query)
	cursor.execute(insert_query)
	cnxn.commit()



Last Record in DB: 2021-01-01 00:00:00
New Records: 10188
Inserting Slowly... INSERT INTO power_demand (date, hour_zulu, state, demand_mgwhr, dt_loaded) VALUES ('2022-01-01 00:00:00', '0',' Texas', '45943', '2022-03-01')
Inserting Slowly... INSERT INTO power_demand (date, hour_zulu, state, demand_mgwhr, dt_loaded) VALUES ('2022-01-01 01:00:00', '1',' Texas', '46427', '2022-03-01')
Inserting Slowly... INSERT INTO power_demand (date, hour_zulu, state, demand_mgwhr, dt_loaded) VALUES ('2022-01-01 02:00:00', '2',' Texas', '45058', '2022-03-01')
Inserting Slowly... INSERT INTO power_demand (date, hour_zulu, state, demand_mgwhr, dt_loaded) VALUES ('2022-01-01 03:00:00', '3',' Texas', '43430', '2022-03-01')
Inserting Slowly... INSERT INTO power_demand (date, hour_zulu, state, demand_mgwhr, dt_loaded) VALUES ('2022-01-01 04:00:00', '4',' Texas', '42058', '2022-03-01')
Inserting Slowly... INSERT INTO power_demand (date, hour_zulu, state, demand_mgwhr, dt_loaded) VALUES ('2022-01-01 05:00:00', '